In [1]:
import numpy as np
import pandas as pd

In [2]:
df_IMDB_Akas = pd.read_csv("./Prj_Data/DownLoadedData_Imdb/title.akas (1).tsv.gz", delimiter='\t', encoding='ANSI')
IM_title_Basics = pd.read_csv("./Prj_Data/DownLoadedData_Imdb/title.basics (1).tsv.gz", delimiter='\t', encoding='ANSI')
df_MetaDataFromFinanceTables = pd.read_excel("./Prj_Data/ImdbScrapingData/df_MasterFinancials_MetaData.xlsx")

C:\Users\rgpih\anaconda3a\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\rgpih\anaconda3a\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
#This table is used as a filter, filter on english and production type
df_IMDB_Akas = df_IMDB_Akas.loc[(df_IMDB_Akas['language']=='en')]
df_IMDB_Akas = df_IMDB_Akas.loc[(df_IMDB_Akas['types']=='imdbDisplay')]

In [4]:
df_IMDB_Akas = df_IMDB_Akas.drop_duplicates(subset='titleId', keep='first').drop(['types', 'attributes',"isOriginalTitle", 'region', "language", "ordering", "title"], axis=1)

IM_title_Basics = IM_title_Basics.loc[(IM_title_Basics['isAdult']==0)]
IM_title_Basics.drop(['isAdult', 'endYear', 'titleType', 'originalTitle'], axis=1, inplace=True)
IM_title_Basics["runtimeMinutes"] = IM_title_Basics.runtimeMinutes.replace(r'\N',0, regex=False)
IM_title_Basics['runtimeMinutes'] = IM_title_Basics['runtimeMinutes'].astype(int)
IM_title_Basics = IM_title_Basics.loc[(IM_title_Basics['runtimeMinutes']>60)]
IM_title_Basics["startYear"] = IM_title_Basics.startYear.replace(r'\N',0, regex=False)
IM_title_Basics['startYear'] = IM_title_Basics['startYear'].astype(int)
IM_title_Basics = IM_title_Basics.loc[(IM_title_Basics['startYear']>=2005) & (IM_title_Basics['startYear']<=2020)]
IM_title_Basics['startYear_str'] = IM_title_Basics['startYear'].astype(str)
IM_title_Basics['primaryTitle'] = IM_title_Basics['primaryTitle'].str.title()
IM_title_Basics['titleyear'] = IM_title_Basics['primaryTitle'] + IM_title_Basics['startYear_str']

In [5]:
df_IMDB_Akas_english = IM_title_Basics.merge(df_IMDB_Akas, left_on="tconst",right_on="titleId",indicator='ind_Link', how="inner")

In [6]:
df_IMDB_Akas_english.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31060 entries, 0 to 31059
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   tconst          31060 non-null  object  
 1   primaryTitle    31060 non-null  object  
 2   startYear       31060 non-null  int32   
 3   runtimeMinutes  31060 non-null  int32   
 4   genres          31060 non-null  object  
 5   startYear_str   31060 non-null  object  
 6   titleyear       31060 non-null  object  
 7   titleId         31060 non-null  object  
 8   ind_Link        31060 non-null  category
dtypes: category(1), int32(2), object(6)
memory usage: 1.9+ MB


In [7]:
df_IMDB_Akas_english.drop(["ind_Link"], axis=1, inplace=True)

In [8]:
df_IMDB_Akas_english.head()

,tconst,primaryTitle,startYear,runtimeMinutes,genres,startYear_str,titleyear,titleId
0,tt0069049,The Other Side Of The Wind,2018,122,Drama,2018,The Other Side Of The Wind2018,tt0069049
1,tt0100275,The Wandering Soap Opera,2017,80,"Comedy,Drama,Fantasy",2017,The Wandering Soap Opera2017,tt0100275
2,tt0118141,What Is It?,2005,72,Drama,2005,What Is It?2005,tt0118141
3,tt0119970,My Apocalypse,2008,82,"Comedy,Drama,Sci-Fi",2008,My Apocalypse2008,tt0119970
4,tt0120667,Fantastic Four,2005,106,"Action,Adventure,Fantasy",2005,Fantastic Four2005,tt0120667


In [9]:
df_MetaDataFromFinanceTables.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5543 entries, 0 to 5542
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Unnamed: 0   5543 non-null   int64         
 1   tconst       5543 non-null   object        
 2   studio_BOM   5543 non-null   object        
 3   genres       4920 non-null   object        
 4   MPAA         3315 non-null   object        
 5   title        5543 non-null   object        
 6   year         5543 non-null   int64         
 7   RunningTime  5543 non-null   int64         
 8   rd           5543 non-null   datetime64[ns]
 9   titleyear    5543 non-null   object        
 10  StudioDesc   4962 non-null   object        
dtypes: datetime64[ns](1), int64(3), object(7)
memory usage: 476.5+ KB


In [10]:
df_MetaDataFromFinanceTables.drop(["year","RunningTime", "genres", "title","titleyear"], inplace=True, axis=1)

In [11]:
df_IMDB_Eng_with_metadata = df_IMDB_Akas_english.merge(df_MetaDataFromFinanceTables, on="tconst",indicator='Merg_MetaData', how="left")

In [12]:
df_IMDB_Eng_with_metadata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31060 entries, 0 to 31059
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   tconst          31060 non-null  object        
 1   primaryTitle    31060 non-null  object        
 2   startYear       31060 non-null  int32         
 3   runtimeMinutes  31060 non-null  int32         
 4   genres          31060 non-null  object        
 5   startYear_str   31060 non-null  object        
 6   titleyear       31060 non-null  object        
 7   titleId         31060 non-null  object        
 8   Unnamed: 0      4560 non-null   float64       
 9   studio_BOM      4560 non-null   object        
 10  MPAA            2865 non-null   object        
 11  rd              4560 non-null   datetime64[ns]
 12  StudioDesc      4057 non-null   object        
 13  Merg_MetaData   31060 non-null  category      
dtypes: category(1), datetime64[ns](1), float64(1), int32(2

In [13]:
df_IMDB_Eng_with_metadata.drop(["titleId","Unnamed: 0","Merg_MetaData" ], axis=1, inplace=True)

In [14]:
df_IMDB_Eng_with_metadata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31060 entries, 0 to 31059
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   tconst          31060 non-null  object        
 1   primaryTitle    31060 non-null  object        
 2   startYear       31060 non-null  int32         
 3   runtimeMinutes  31060 non-null  int32         
 4   genres          31060 non-null  object        
 5   startYear_str   31060 non-null  object        
 6   titleyear       31060 non-null  object        
 7   studio_BOM      4560 non-null   object        
 8   MPAA            2865 non-null   object        
 9   rd              4560 non-null   datetime64[ns]
 10  StudioDesc      4057 non-null   object        
dtypes: datetime64[ns](1), int32(2), object(8)
memory usage: 2.6+ MB


In [15]:
df_IMDB_Eng_with_metadata.to_excel("df_IMDB_MovieCatalog.xlsx")

Data belwo is for quick cutting and copying for testing :)

In [ ]:
# df_IMDB_Akas_english.drop(["ind_Link","Unnamed: 0","titleyear_fin", "titleId","genres_fin","MPAA_fin",
#                           "title_fin","Merg_MetaData", "year_fin", "RunningTime_fin"], axis=1, inplace=True)

# df_IMDB_Akas_english.drop(["ind_Link","Unnamed: 0","titleyear_fin", "titleId","genres_fin","MPAA_fin",
#                           "title_fin","Merg_MetaData", "year_fin", "RunningTime_fin"], axis=1, inplace=True)

In [ ]:
testingLink.loc[testingLink['tconst'] == "tt0499549"][["primaryTitle_x", "tconst"]]
# .loc[df_IMDB_Akas['tconst'] == "tt0499549"][["primaryTitle", "tconst"]]
# df_IMDB_Akas_english[df_IMDB_Akas_english.primaryTitle.str.contains('Happy Together', case=False)]